In [44]:
import pandas as pd
import sys
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import time

In [ ]:
limit = 8 # setting to 3 articles limit per page

def getDate(url):
    try :
        req = Request(
            url=url, 
            headers={'User-Agent': 'Mozilla/5.0'}
        )
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, 'html.parser')
        dates = soup.find_all(class_ = "date") 
        dateArray=[]
        count = 1
        for date in dates :
            d = date.get_text().split()[1:]
            if d :
                dateArray.append(d)
            else :
                break
            if count >= limit :
                break
            count += 1    
        return (dateArray,count)
    except :
        print("error")


# tweak start and end for faster compile
def getStartEnd(url,startnum,endnum):
    try :
        check = True
        start,count = getDate("{}{}".format(url,startnum))
        while check :
            if (start[0][0] == 'Aug' and start[0][2] == '2022') :
                check = False
                break
            startnum += 1
            start,count = getDate("{}{}".format(url,startnum))
            print(start)
        print(startnum)
        # coronavirus special - oldest date to mar 2020
        if url == 'https://www.investing.com/news/coronavirus/' :
            endnum = 650 # last updated 17 October 2022
        else :
            check = True
            end,count = getDate("{}{}".format(url,endnum))
            while check :
                if end[0][0] == 'Jan' and end[0][2] == '2018' :
                    check = False
                    break
                endnum += 1
                end,count = getDate("{}{}".format(url,endnum))    
                print(end)
            print(endnum)
        return (startnum,endnum)
    except : 
        print("error")

def getArticles(url,startnum,endnum):
    df = pd.DataFrame(columns=['Date','Title'])
    try :
        start,end = getStartEnd(url,startnum,endnum)
        while start <= end :
            req = Request(
                url="{}{}".format(url,start), 
                headers={'User-Agent': 'Mozilla/5.0'}
            )
            webpage = urlopen(req).read() 
            soup = BeautifulSoup(webpage, 'html.parser')
            dates = soup.find_all(class_="date")
            titles = soup.find_all(class_ = "title") 
            texts = soup.find_all("p")
            nTexts = []
            textCount = 0
            for t in texts :
                if t.text[:2] == "By":
                    nTexts.append(t.text)
                    textCount += 1
                if textCount == limit :
                    break
            for i in range(limit) :
                d = dates[i].get_text().split()[1:]
                if d :
                    df_new = pd.DataFrame({"Date" : dates[i].text[3:], "Title" : titles[i].text},index=[1])
                    df = pd.concat([df,df_new],ignore_index=True)
                else :
                    break
            print(start, end, "[Page {}/{}]".format(start, end))
            start += 1
            time.sleep(0.5)
        return df
    except :
        print("error")

In [47]:
df_tsla = getArticles("https://www.investing.com/equities/tesla-motors-news/",40,815)
df_tsla.to_csv('../Data/News/Stock/tsla_news.csv',index=False)

[['Aug', '31,', '2022'], ['Aug', '31,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022']]
41
[['Jan', '29,', '2018'], ['Jan', '28,', '2018'], ['Jan', '26,', '2018'], ['Jan', '25,', '2018'], ['Jan', '25,', '2018'], ['Jan', '25,', '2018'], ['Jan', '24,', '2018'], ['Jan', '24,', '2018']]
816
41 816 [Page 41/816]
42 816 [Page 42/816]
43 816 [Page 43/816]
44 816 [Page 44/816]
45 816 [Page 45/816]
46 816 [Page 46/816]
47 816 [Page 47/816]
48 816 [Page 48/816]
49 816 [Page 49/816]
50 816 [Page 50/816]
51 816 [Page 51/816]
52 816 [Page 52/816]
53 816 [Page 53/816]
54 816 [Page 54/816]
55 816 [Page 55/816]
56 816 [Page 56/816]
57 816 [Page 57/816]
58 816 [Page 58/816]
59 816 [Page 59/816]
60 816 [Page 60/816]
61 816 [Page 61/816]
62 816 [Page 62/816]
63 816 [Page 63/816]
64 816 [Page 64/816]
65 816 [Page 65/816]
66 816 [Page 66/816]
67 816 [Page 67/816]
68 816 [Page 68/816]
69 816 [Page 69/816

In [48]:
df_aapl = getArticles("https://www.investing.com/equities/apple-computer-inc-news/",30,1034) 
print(df_aapl.head())
print(df_aapl.shape)
df_aapl.to_csv('../Data/News/Stock/aapl_news.csv',index=False)

[['Sep', '06,', '2022'], ['Sep', '05,', '2022'], ['Sep', '04,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022']]
[['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '01,', '2022'], ['Sep', '01,', '2022'], ['Sep', '01,', '2022']]
[['Aug', '31,', '2022'], ['Aug', '31,', '2022'], ['Aug', '31,', '2022'], ['Aug', '31,', '2022'], ['Aug', '31,', '2022'], ['Aug', '31,', '2022'], ['Aug', '31,', '2022'], ['Aug', '30,', '2022']]
33
[['Feb', '02,', '2018'], ['Feb', '02,', '2018'], ['Feb', '02,', '2018'], ['Feb', '02,', '2018'], ['Feb', '02,', '2018'], ['Feb', '02,', '2018'], ['Feb', '02,', '2018'], ['Feb', '02,', '2018']]
[['Feb', '01,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018']]
[['Feb', '01,', '2018'], ['Feb',

In [50]:
df_meta = getArticles("https://www.investing.com/equities/facebook-inc-news/",20,1211) 
print(df_meta.head())
print(df_meta.shape)
df_meta.to_csv('../Data/News/Stock/meta_news.csv',index=False)

[['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '01,', '2022'], ['Sep', '01,', '2022'], ['Aug', '31,', '2022'], ['Aug', '31,', '2022']]
[['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '29,', '2022'], ['Aug', '29,', '2022']]
22
[['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018']]
1212
22 1212 [Page 22/1212]
23 1212 [Page 23/1212]
24 1212 [Page 24/1212]
25 1212 [Page 25/1212]
26 1212 [Page 26/1212]
27 1212 [Page 27/1212]
28 1212 [Page 28/1212]
29 1212 [Page 29/1212]
30 1212 [Page 30/1212]
31 1212 [Page 31/1212]
32 1212 [Page 32/1212]
33 1212 [Page 33/1212]
34 1212 [Page 34/1212]
35 1212 [Page 35/1212]
36 1212 [Page 36/1212]
37 1212 [Page 37/1212]
38 1212 [Page 38/1212]
39 1212 [Page 39/1212]

In [52]:
df_world = getArticles("https://www.investing.com/news/world-news/",98,4313)
print(df_world.head())
print(df_world.shape)
df_world.to_csv('../Data/News/Global/world_news.csv',index=False)

[['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022']]
[['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022']]
[['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '01,', '2022']]
[['Sep', '01,', '2022'], ['Sep', '01,', '2022'], ['Sep', '01,', '2022'], ['Sep', '01,', '2022'], ['Sep', '01,', '2022'], ['Sep', '01,', '2022'], ['Sep', '01,', '2022'], ['Sep', '01,', '2022']]
[['Sep', '01,', '2022'], ['Sep', '01,', '2022'], ['Sep', '01,', '2022'], ['Sep', '01,', '2022'], ['Sep', '01,', '2022'], ['Sep', '01,', '2022'], ['Sep', '01,', '2022'], ['Sep', '01,', '2022']]
[['Sep', '01,', '2022'], ['Sep', '0

AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:
df_politics = getArticles("https://www.investing.com/news/politics/",9,780) 
print(df_politics.head())
print(df_politics.shape)
df_politics.to_csv('../Data/News/Global/politics_news.csv',index=False)

In [ ]:
df_coronavirus = getArticles("https://www.investing.com/news/coronavirus/",3,650) 
print(df_coronavirus.head())
print(df_coronavirus.shape)
df_coronavirus.to_csv('../Data/News/Global/coronavirus_news.csv',index=False)